This demo notebook shows how to use the routines in pbe.py to: 
- export a matrix $W$ in homogeneous coordinates
- launch the Polymake script "vertex-facet.pl" to compute the vertex-facet incidences for $W$
- apply the polytope bias estimation (PBE) on the ball to obtain $\alpha^\mathbb{B}$
- reconstruct $x$ from $\operatorname{ReLU}(Wx-\alpha^\mathbb{B})$

In [1]:
import numpy as np
import pbe

Examples: Icosahedron and random frame on the sphere

In [2]:
phi = (1+np.sqrt(5))/2
ico = np.array([[0,1,phi],[0,1,-phi],[0,-1,phi],[0,-1,-phi],
                [1,phi,0],[-1,phi,0],[1,-phi,0],[-1,-phi,0],
                [phi,0,1],[phi,0,-1],[-phi,0,1],[-phi,0,-1]])/(np.sqrt(1+phi**2))
ran = pbe.norm_row(np.random.randn(12,3))[0]

Export in homogeneous coordiantes as .txt file

In [3]:
pbe.hom(ico, filename="ico")
pbe.hom(ran, filename="ran")

Open polymake via bash and run the script "vertex-facet.pl" with argument "/MyDir/ico".\
This will create "/MyDir/ico_facets.csv" containing the vertex-facet incidences. Use pbe.read_facets("/MyDir/ico_facets.csv") to get them explicitely in an array where the k-th row corresponds to the vertex-indices of the k-th facet.\
\
**!!! if vertex-facet incidences are computed in a loop it is recommended to do the loop inside the .pl script !!!**

In [10]:

!polymake --script "vertex-facet.pl" "/MyDir/ico"
!polymake --script "vertex-facet.pl" "/MyDir/ran"

polymake: used package ppl
  The Parma Polyhedra Library ([[wiki:external_software#PPL]]): A C++ library for convex polyhedra
  and other numerical abstractions.
  http://www.cs.unipr.it/ppl/

polymake: used package ppl
  The Parma Polyhedra Library ([[wiki:external_software#PPL]]): A C++ library for convex polyhedra
  and other numerical abstractions.
  http://www.cs.unipr.it/ppl/



PBE on $\mathbb{B}$: get the upper bias $\alpha^\mathbb{B}$

In [5]:
alpha_ico = pbe.pbe(ico, filename="/MyDir/ico_facets.csv", radius=1)
alpha_ran = pbe.pbe(ran, filename="/MyDir/ran_facets.csv", radius=1)
print('alpha^B for the Icosahedron frame:', alpha_ico)
print('alpha^B for a random frame:', alpha_ran)

alpha^B for the Icosahedron frame: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
alpha^B for a random frame: [-0.2823 -0.2823  0.      0.      0.      0.      0.      0.     -0.1446
 -0.1446 -0.1446 -0.1446]


Reconstruction of a random vector

In [8]:
x = np.random.randn(3)
x = x/np.linalg.norm(x)
# output of the ReLU-layer
z = pbe.relu(x, ran, alpha_ran)
# recostruct x
x_hat = pbe.relu_inv(z, ran, alpha_ran, "/MyDir/ran_facets.csv")
print('Error:', np.linalg.norm(x-x_hat))

Facet 3 with vertices [0, 4, 8] is used for reconstruction.
Error: 8.50064065311604e-16
